In [4]:
import os
import pandas as pd
import warnings
from tabulate import tabulate
from dateutil import parser
import re
from datetime import datetime, timedelta
import numpy as np

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Suppress the SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Define paths
main_path = r'C:\Users\Loop\Desktop\Process Tools\Endo Recon Tool\Input\Main'
sub_path = r'C:\Users\Loop\Desktop\Process Tools\Endo Recon Tool\Input\Sub'
output_path = r'C:\Users\Loop\Desktop\Process Tools\Endo Recon Tool\Output'

# Helper function to read files from a given path
def read_files_from_path(path):
    data_frames = []
    for file_name in os.listdir(path):
        if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
            df = pd.read_excel(os.path.join(path, file_name), None)  # Read all sheets
            data_frames.append((file_name, df))
        elif file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(path, file_name))
            data_frames.append((file_name, df))
    return data_frames

# Read files from main and sub paths
main_data_frames = read_files_from_path(main_path)
sub_data_frames = read_files_from_path(sub_path)

# Check if any files were found in both paths
if not main_data_frames and not sub_data_frames:
    print("No files found in both Main and Sub directories. Process aborted.")
elif not main_data_frames:
    print("No files found in Main directory. Process aborted.")
elif not sub_data_frames:
    print("No files found in Sub directory. Process aborted.")
else:
    def process_data_frames(data_frames, data_type="main"):
        selected_data = {}
    
        for file_name, dfs in data_frames:
            if isinstance(dfs, dict):  # Check if it's a dictionary of sheets
                sheet_names = list(dfs.keys())
                if len(sheet_names) > 1:
                    print(f"File '{file_name}' contains multiple sheets: {sheet_names}")
                    selected_sheet = input(f"Please enter the sheet name you want to process from '{file_name}': ")
                    if selected_sheet in sheet_names:
                        df = dfs[selected_sheet]
                        selected_data[file_name] = df
                        print(f"Processing sheet '{selected_sheet}' from file '{file_name}'")
                    else:
                        print(f"Sheet '{selected_sheet}' not found in file '{file_name}'. Process aborted.")
                else:
                    df = dfs[sheet_names[0]]  # Only one sheet, no need to ask
                    selected_data[file_name] = df
                    print(f"Processing single sheet '{sheet_names[0]}' from file '{file_name}'")
            else:
                df = dfs  # It's a CSV file
                selected_data[file_name] = df
                print(f"Processing CSV file '{file_name}'")

        for file_name, df in selected_data.items():
            required_columns = ["Employee ID", "Name", "Relationship", "DOB", "Gender", "Coverage Start Date", "Sum Insured", "UHID", "Active","Phone","Email"]
            df.columns = [col.strip() for col in df.columns]  # Strip any extra spaces
            missing_columns = [col for col in required_columns if col not in df.columns]

            if missing_columns:
                # Printing DataFrame in Tabular format
                pd.set_option("display.max_columns", None)
                display(df.head())

                # Print missing and available columns using tabulate in JIRA format
                print(f"\nColumns missing in '{file_name}': {missing_columns}.\n\nAvailable columns:\n")
                print(tabulate([df.columns], headers='firstrow', tablefmt='jira'))
                print("\n")

                for missing in missing_columns:
                    attempt = 0
                    while attempt < 2:
                        column_name = input(f"Please input the column name for '{missing}': ").strip()
                        if column_name in df.columns:
                            df.rename(columns={column_name: missing}, inplace=True)
                            break
                        else:
                            print("Invalid column name. Please try again.")
                            attempt += 1
                    if attempt == 2:
                        print(f"Skipping renaming for missing column '{missing}' after 2 invalid attempts.")
            else:
                print(f"All required columns are present in file '{file_name}'.")

        return selected_data
    
    
    def clean_and_validate_data(selected_data):
        # Function to convert date strings to the desired format
        def convert_to_dd_mmm_yyyy(date_str):
            if pd.isnull(date_str):
                return None
            try:
                # Parse the date using dateutil.parser
                date = parser.parse(date_str)
                # Format the date to 'DD-MMM-YYYY'
                return date.strftime('%d/%b/%Y')
            except Exception as e:
                print(f"Error parsing date: {date_str}. Exception: {e}")
                return None  # or handle the exception as needed

        def update_relationship(row):
            relationship = row['Relationship'].strip().title()  # Remove leading and trailing spaces
            gender = row['Gender'].strip().lower()
            if relationship in ['Self', 'Employee']:
                return 'Self'
            elif relationship in ['Parent', 'Mother', 'Father']:
                return 'Father' if gender == 'male' else 'Mother'
            elif relationship in ['Spouse', 'Wife', 'Husband']:
                return 'Husband' if gender == 'male' else 'Wife'
            elif relationship in ['Child', 'Son', 'Daughter']:
                return 'Son' if gender == 'male' else 'Daughter'
            return relationship
        
        def preprocess_mobile_number(number):
            if pd.isna(number):
                return number
            str_number = str(number).strip()  # Convert to string and strip any surrounding whitespace
            if str_number.endswith('.0'):
                str_number = str_number[:-2]  # Remove trailing '.0' if present
            elif str_number.startswith('+91'):
                str_number = str_number[3:]
        
            # Remove any internal spaces and return the number if it only contains digits
            str_number = str_number.replace(' ', '')  # Remove any spaces within the number

            return str_number if str_number.isdigit() else number  # Return as string if it's a valid number
        
        
        # Convert 'Relation' column values
        def convert_relation(relation):
            if relation == 'Employee':
                 return 'Self'
            elif relation in ['Wife', 'Husband']:
                 return 'Spouse'
            elif relation in ['Daughter', 'Son']:
                return 'Child'
            elif relation in ['Father', 'Mother']:
                 return 'Parent'
            elif relation in ['Father In Law', 'Mother In Law']:
                 return 'Parent-in-law'
            else:
                return relation
            
            
        def update_gender(row):
            Gender = row['Gender'].strip().title()  # Remove leading and trailing spaces
            if Gender in ['Male', 'M']:
                return 'Male'
            elif Gender in ['Female', 'F']:
                return 'Female' 
            return Gender


        # Process the data in each file
        for file_name, df in selected_data.items():
            if df['Employee ID'].dtype == 'object':
                df['Employee ID'] = df['Employee ID'].str.replace(r'\s+', '', regex=True)
            else:
                # Handle NaN or inf values before converting to int
                df['Employee ID'] = df['Employee ID'].replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)

            if 'Name' in df.columns:
                df['Name'] = df['Name'].fillna('')
                df['Name'] = df['Name'].astype(str).str.lower()
                titles_to_remove = ['mr', 'mrs', 'master', 'dr', 'miss']
                title_pattern = r'\b(?:' + '|'.join(titles_to_remove) + r')\b'
                special_chars_pattern = r'[!@#$%^&*()_+\-=\[\]{}\\|:";\'<>?,./1234567890]'
                df['Name'] = (df['Name']
                              .str.replace(title_pattern, '', regex=True)
                              .str.replace(special_chars_pattern, '', regex=True)
                              .str.replace(r'\s+', ' ', regex=True)
                              .str.strip()
                              .str.title())
                df['Name'] = df['Name'].replace('', np.nan)

            if 'Gender' in df.columns:
                df['Gender'] = df.apply(update_gender, axis=1)

            if 'DOB' in df.columns:
                df['DOB'] = pd.to_datetime(df['DOB']).dt.strftime('%d/%b/%Y')
                df['DOB'] = df['DOB'].apply(lambda x: convert_to_dd_mmm_yyyy(x) if pd.notnull(x) else None)
                df['DOB'] = df['DOB'].astype(str).str.strip().str.replace(r'[\u200B-\u200D\uFEFF]', '', regex=True)

            if 'DOB' in df.columns:
                df['Age'] = pd.to_datetime(df['DOB'], format='mixed', dayfirst=True, errors='coerce')
                df['Age'] = ((datetime.now() - df['Age']).dt.days // 365).fillna(0).astype(int)
                cols = df.columns.tolist()
                dob_index = cols.index('DOB')
                cols.insert(dob_index + 1, cols.pop(cols.index('Age')))
                df = df[cols]

            if 'Relationship' in df.columns:
                df['Relationship'] = df.apply(update_relationship, axis=1)
         
            # Reorder columns to have 'Relationship' next to 'Name'
            if 'Relationship' in df.columns:
                cols = df.columns.tolist()
                relation_index = cols.index('Name')
                cols.insert(relation_index + 1, cols.pop(cols.index('Relationship')))
                df = df[cols]
                              

            df['Relation Type'] = df['Relationship'].apply(convert_relation)
    
            # Reorder columns to have 'Relation Type' next to 'Relation'
            cols = df.columns.tolist()
            relation_index = cols.index('Relationship')
            cols.insert(relation_index + 1, cols.pop(cols.index('Relation Type')))
            df = df[cols]


            if 'Coverage Start Date' in df.columns:
                df['Coverage Start Date'] = df['Coverage Start Date'].astype(str).str.strip().apply(lambda x: convert_to_dd_mmm_yyyy(x) if x else '')

            if 'Sum Insured' in df.columns:
                # Remove whitespace and ensure the value is a string without float formatting
                df['Sum Insured'] = df['Sum Insured'].astype(str).str.replace(r'\s+', '', regex=True).str.split('.').str[0]


            if 'Employee ID' in df.columns and 'Relationship' in df.columns and 'Coverage Start Date' in df.columns:
                for emp_id in df.loc[df['Relationship'].isin(['Self', 'Employee']), 'Employee ID'].unique():
                    coverage_start_date = df.loc[(df['Employee ID'] == emp_id) & (df['Relationship'] == 'Self'), 'Coverage Start Date'].iloc[0]
                    df.loc[df['Employee ID'] == emp_id, 'Coverage Start Date'] = coverage_start_date

            if 'Employee ID' in df.columns and 'Relationship' in df.columns and 'Sum Insured' in df.columns:
                for emp_id in df.loc[df['Relationship'].isin(['Self', 'Employee']), 'Employee ID'].unique():
                    sum_insured = df.loc[(df['Employee ID'] == emp_id) & (df['Relationship'] == 'Self'), 'Sum Insured'].iloc[0]
                    df.loc[df['Employee ID'] == emp_id, 'Sum Insured'] = sum_insured
                    
            # Assuming df is your DataFrame
            if 'UHID' in df.columns:
                df['UHID'] = df['UHID'].astype(str).str.replace(r'\s+', '', regex=True).str.split('.').str[0]
                df['UHID'] = df['UHID'].replace(['nan', ''], '')
                
                
            # Mapping dictionary
            mapping = {
                'Yes': 'Yes', 'Y': 'Yes', 'Active': 'Yes', 'A': 'Yes', 'M': 'Yes', 'ACTIVE': 'Yes', 'YES': 'Yes',
                'No': 'No', 'N': 'No', 'In-Active': 'No', 'In active': 'No', 'Delete': 'No', 'D': 'No', 
                'Deleted/Cancelled': 'No', 'IN-ACTIVE': 'No', 'DELETE': 'No', 'NO': 'No', 'CANCELLED': 'No'
            }

            # Check if 'Active' column exists
            if 'Active' in df.columns:
                df['Active'] = df['Active'].astype(str).str.strip().replace('', np.nan)
                df['Active'] = df['Active'].map(mapping).fillna(df['Active'])
            else:
                df['Active'] = np.nan
                            

            # Apply preprocessing to the 'Phone' column
            if 'Phone' in df.columns:
                df['Phone'] = df['Phone'].apply(preprocess_mobile_number)
                
            # Clean the 'Email' column
            if 'Email' in df.columns:
                
                if not df['Email'].isna().all():
                    df['Email'] = df['Email'].str.lower()

                 
            selected_data[file_name] = df
        return selected_data
    
    
    def compare_and_filter_employee_ids(selected_main_data, selected_sub_data):
        main_employee_ids = set()
        sub_employee_ids = set()

        # Collect all Employee IDs from main data
        for file_name, df in selected_main_data.items():
            if 'Employee ID' in df.columns:
                main_employee_ids.update(df['Employee ID'].unique())

        # Collect all Employee IDs from sub data
        for file_name, df in selected_sub_data.items():
            if 'Employee ID' in df.columns:
                sub_employee_ids.update(df['Employee ID'].unique())

        # Identify mismatches
        sub_not_in_main = sub_employee_ids - main_employee_ids

        # Create unmatch_sub_data and update selected_sub_data
        unmatch_sub_data = {}
        for file_name, df in selected_sub_data.items():
            if 'Employee ID' in df.columns:
                # Identify unmatched rows
                unmatched_rows = df[df['Employee ID'].isin(sub_not_in_main)]
                if not unmatched_rows.empty:
                    unmatch_sub_data[file_name] = unmatched_rows
            
                # Update selected_sub_data to keep only matching rows
                matched_rows = df[~df['Employee ID'].isin(sub_not_in_main)]
                selected_sub_data[file_name] = matched_rows

        return unmatch_sub_data, selected_sub_data
    
#-----------------------------------------------------------------------------------------------------------------------------    
       
    
    # Function to save data to Excel
    def save_to_excel(selected_main_data, Unmatched_main_data, selected_sub_data, unmatch_sub_data,  output_path):
        output_file = os.path.join(output_path, 'Recon Data.xlsx')
    
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # Save selected_main_data to "Main Data" sheet
            sheet_name_main = 'Main Data'
            for file_name, df in selected_main_data.items():
                df.to_excel(writer, sheet_name=sheet_name_main, index=False)
                
            # Save Unmatched_main_data to "Unmatched Main Data" sheet
            sheet_name_unmatched_main = 'Unmatched Main Data'
            for file_name, df in Unmatched_main_data.items():
                df.to_excel(writer, sheet_name=sheet_name_unmatched_main, index=False)
                
             # Save unmatch_sub_data to "Unmatched Sub Data" sheet
            sheet_name_matched = 'selected Sub Data'
            for file_name, df in selected_sub_data.items():
                df.to_excel(writer, sheet_name=sheet_name_matched, index=False)
        
            # Save unmatch_sub_data to "Unmatched Sub Data" sheet
            sheet_name_unmatched = 'Unmatched Sub Data'
            for file_name, df in unmatch_sub_data.items():
                df.to_excel(writer, sheet_name=sheet_name_unmatched, index=False)
                

        print(f"Recon Data saved to {output_file}")

    # Process main data frames
    selected_main_data = process_data_frames(main_data_frames, data_type="main")

    # Process sub data frames
    selected_sub_data = process_data_frames(sub_data_frames, data_type="sub")

    # Clean and validate main data
    selected_main_data = clean_and_validate_data(selected_main_data)

    # Clean and validate sub data
    selected_sub_data = clean_and_validate_data(selected_sub_data)

    # Compare Employee IDs between main and sub data and filter sub data
    unmatch_sub_data, selected_sub_data = compare_and_filter_employee_ids(selected_main_data, selected_sub_data)

    
    #------------------------------------------------------------------------------------------------------------------------
    
   # Create a temporary column by concatenating 'Employee ID' (converted to string) and 'Name'
    for file_name, df in selected_main_data.items():
        if 'Employee ID' in df.columns and 'Name' in df.columns:
            df['Temp_Column'] = df['Employee ID'].astype(str).str.replace(r'[^a-zA-Z0-9]', '', regex=True).str.upper() + " " + df['Name']
    
        # To fill any blank value in df['Temp_Column']
        df['Temp_Column'] = df['Temp_Column'].fillna('Unknown')
    
    # Create a temporary column by concatenating 'Employee ID' and 'Name' for sub data
    for file_name, comparison_df in selected_sub_data.items():
        if 'Employee ID' in comparison_df.columns and 'Name' in comparison_df.columns:
            comparison_df['Temp_Column'] = comparison_df['Employee ID'].astype(str).str.replace(r'[^a-zA-Z0-9]', '', regex=True).str.upper() + " " + comparison_df['Name']
        
        # To fill any blank value in df['Temp_Column']
        comparison_df['Temp_Column'] = comparison_df['Temp_Column'].fillna('Unknown')
    
    
    # Getting User ID and UHID column from Genome data
    # Columns 
    Relationship_column = "Relationship"
    DOB_column = "DOB"
    Gender_column = "Gender"
    Sum_Insured_column = "Sum Insured"
    Coverage_Start_Date_column = "Coverage Start Date"
    UHID_column = "UHID"
    active_column = 'Active'
    phone_column = 'Phone'
    email_column = 'Email'


    # Check if the columns exist in the comparison_df
    Relationship_exists = Relationship_column in comparison_df.columns
    DOB_exists = DOB_column in comparison_df.columns
    Gender_exists = Gender_column in comparison_df.columns
    Sum_Insured_exists = Sum_Insured_column in comparison_df.columns
    Coverage_Start_Date_exists = Coverage_Start_Date_column in comparison_df.columns
    uhid_exists = UHID_column in comparison_df.columns
    active_exists = active_column in comparison_df.columns
    phone_exists = phone_column in comparison_df.columns
    email_exists = email_column in comparison_df.columns


    def match_names(df_temp, comp_temp, comp_uhid=None, comp_active=None, comp_Relationship=None, comp_DOB=None, comp_Gender=None, comp_Sum_Insured=None, comp_Coverage_Start_Date=None, comp_phone=None, comp_email=None):
        
        df_parts = df_temp.split()
        comp_parts = comp_temp.split()

        # Check if Employee ID matches
        if df_parts[:2] != comp_parts[:2]:
            return False, None, None, None, None, None, None, None, None, None

        # Compare the names
        df_name_parts = set(df_parts[2:])
        comp_name_parts = set(comp_parts[2:])

        # Finding common name parts
        common_parts = df_name_parts.intersection(comp_name_parts)

        # We could also consider common_parts >= 1 if we want to be less strict
        if len(common_parts) >= 1:
            return True, comp_Relationship, comp_DOB, comp_Gender, comp_Sum_Insured, comp_Coverage_Start_Date, comp_uhid, comp_active, comp_phone, comp_email
        else:
            return False, None, None, None, None, None, None, None, None, None


    def get_matches(df_temp, comparison_df):
        for i, row in comparison_df.iterrows():
            match, active, Relationship, DOB, Gender, Sum_Insured, Coverage_Start_Date, UHID, phone, email,  = match_names(
                df_temp,
                row['Temp_Column'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )
            if match:
                return row['Temp_Column']
        return None

    # Apply the matching function
    df['Match Found Genome'] = df['Temp_Column'].apply(
        lambda x: any(
            match_names(
                x,
                row['Temp_Column'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )[0] for _, row in comparison_df.iterrows()
        )
    )

    # Extra added Start
    df['Found on Genome'] = df['Temp_Column'].apply(
        lambda x: get_matches(x, comparison_df) if any(
            match_names(
                x,
                row['Temp_Column'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )[0] for _, row in comparison_df.iterrows()
        ) else None
    )

    # Apply the matching function and retrieve UHID, Active status, and other specified columns
    def get_matching_info(temp_value):
        for i, row in comparison_df.iterrows():
            match, Relationship, DOB, Gender, Sum_Insured, Coverage_Start_Date, UHID, active, phone, email = match_names(
                temp_value,
                row['Temp_Column'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )
            if match:
                return Relationship, DOB, Gender, Sum_Insured, Coverage_Start_Date, UHID, active, phone, email
        return None, None, None, None, None, None, None, None, None

    df['Sub Relationship'], df['Sub DOB'], df['Sub Gender'], df['Sub Sum Insured'], df['Sub Coverage Start Date'], df['Sub UHID'], df['Sub Active'], df['Sub Phone'], df['Sub Email'] = zip(*df['Temp_Column'].map(get_matching_info))
    
    #-------------------------------------------Code for Unmatched columns-----------------------------------------------
    
    # Filter unmatched data and store it in a dictionary format
    Unmatched_main_data = {file_name: df[df['Match Found Genome'] == False].copy() for file_name, df in selected_main_data.items()}

    # Remove the unmatched rows from selected_main_data
    selected_main_data = {file_name: df[df['Match Found Genome'] == True] for file_name, df in selected_main_data.items()}
    
    # Columns to be removed
    columns_to_remove = [
        'Temp_Column', 'Match Found Genome', 'Found on Genome',
        'Sub Relationship', 'Sub DOB', 'Sub Gender', 'Sub Sum Insured',
        'Sub Coverage Start Date', 'Sub UHID', 'Sub Active', 'Sub Phone','Sub Email'
    ]

    # Remove the specified columns from each DataFrame in Unmatched_main_data
    for file_name, df in Unmatched_main_data.items():
        df.drop(columns=columns_to_remove, inplace=True)
        
        
    # Create a temporary column by concatenating 'Employee ID' (converted to string) and 'Name'
    for file_name, df in Unmatched_main_data.items():
        if 'Employee ID' in df.columns and 'DOB' in df.columns and 'Name' in df.columns:
            df['Temp_Column1'] = df['Employee ID'].astype(str).str.replace(r'[^a-zA-Z0-9]', '', regex=True).str.upper() + " " + df['DOB'] + " " + df['Name']
    
        # To fill any blank value in df['Temp_Column']
        df['Temp_Column1'] = df['Temp_Column1'].fillna('Unknown')
    
    # Create a temporary column by concatenating 'Employee ID' and 'Name' for sub data
    for file_name, comparison_df in selected_sub_data.items():
        if 'Employee ID' in comparison_df.columns and 'Name' in comparison_df.columns:
            comparison_df['Temp_Column1'] = comparison_df['Employee ID'].astype(str).str.replace(r'[^a-zA-Z0-9]', '', regex=True).str.upper() + " " + comparison_df['DOB'] + " " + comparison_df['Name']
        
        # To fill any blank value in df['Temp_Column']
        comparison_df['Temp_Column1'] = comparison_df['Temp_Column1'].fillna('Unknown')
        
        
    def match_names(df_temp, comp_temp, comp_uhid=None, comp_active=None, comp_Relationship=None, comp_DOB=None, comp_Gender=None, comp_Sum_Insured=None, comp_Coverage_Start_Date=None, comp_phone=None, comp_email=None):
        df_parts = df_temp.split()
        comp_parts = comp_temp.split()

        # Check if Employee ID matches
        if df_parts[:2] != comp_parts[:2]:
            return False, None, None, None, None, None, None, None, None, None

        # Compare the names
        df_name_parts = set(df_parts[2:])
        comp_name_parts = set(comp_parts[2:])

        # Finding common name parts
        common_parts = df_name_parts.intersection(comp_name_parts)

        # We could also consider common_parts >= 1 if we want to be less strict
        if len(common_parts) == 1:
            return True, comp_Relationship, comp_DOB, comp_Gender, comp_Sum_Insured, comp_Coverage_Start_Date, comp_uhid, comp_active, comp_phone, comp_email
        else:
            return False, None, None, None, None, None, None, None, None, None


    def get_matches(df_temp, comparison_df):
        for i, row in comparison_df.iterrows():
            match, active, Relationship, DOB, Gender, Sum_Insured, Coverage_Start_Date, UHID, phone, email = match_names(
                df_temp,
                row['Temp_Column1'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )
            if match:
                return row['Temp_Column1']
        return None

    # Apply the matching function
    df['Match Found Genome'] = df['Temp_Column1'].apply(
        lambda x: any(
            match_names(
                x,
                row['Temp_Column1'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )[0] for _, row in comparison_df.iterrows()
        )
    )

    # Extra added Start
    df['Found on Genome'] = df['Temp_Column1'].apply(
        lambda x: get_matches(x, comparison_df) if any(
            match_names(
                x,
                row['Temp_Column1'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )[0] for _, row in comparison_df.iterrows()
        ) else None
    )

    # Apply the matching function and retrieve UHID, Active status, and other specified columns
    def get_matching_info(temp_value):
        for i, row in comparison_df.iterrows():
            match, Relationship, DOB, Gender, Sum_Insured, Coverage_Start_Date, UHID, active, phone, email = match_names(
                temp_value,
                row['Temp_Column1'],
                row[UHID_column] if UHID_column in comparison_df.columns else None,
                row[active_column] if active_column in comparison_df.columns else None,
                row[Relationship_column] if Relationship_column in comparison_df.columns else None,
                row[DOB_column] if DOB_column in comparison_df.columns else None,
                row[Gender_column] if Gender_column in comparison_df.columns else None,
                row[Sum_Insured_column] if Sum_Insured_column in comparison_df.columns else None,
                row[Coverage_Start_Date_column] if Coverage_Start_Date_column in comparison_df.columns else None,
                row[phone_column] if phone_column in comparison_df.columns else None,
                row[email_column] if email_column in comparison_df.columns else None
            )
            if match:
                return Relationship, DOB, Gender, Sum_Insured, Coverage_Start_Date, UHID, active, phone, email
        return None, None, None, None, None, None, None, None, None

    if not df.empty:
        df['Sub Relationship'], df['Sub DOB'], df['Sub Gender'], df['Sub Sum Insured'], df['Sub Coverage Start Date'], df['Sub UHID'], df['Sub Active'], df['Sub Phone'], df['Sub Email'] = zip(*df['Temp_Column1'].map(get_matching_info))
    else:
        print("Unmatched_main_data is empty. No operations to perform.")
    
    def move_matched_rows(unmatched_data, selected_data):
        for file_name, df_unmatched in unmatched_data.items():
            matched_rows = df_unmatched[df_unmatched['Match Found Genome'] == True]
            for index, row in matched_rows.iterrows():
                # Move the row directly without checking if Employee ID is in selected_data
                selected_data[file_name] = pd.concat([selected_data[file_name], pd.DataFrame([row])], ignore_index=True)
                unmatched_data[file_name].drop(index, inplace=True)

            # Sort the dataframes after moving rows
            unmatched_data[file_name] = unmatched_data[file_name].sort_values(by=['Employee ID']).reset_index(drop=True)
            selected_data[file_name] = selected_data[file_name].sort_values(by=['Employee ID']).reset_index(drop=True)

    # Apply the function
    move_matched_rows(Unmatched_main_data, selected_main_data)
    
    # Iterate over items in selected_main_data and create 'Match' column
    for file_name, df in selected_main_data.items():
        df['Match Relationship'] = df['Relationship'] == df['Sub Relationship']
        df['Match Gender'] = df['Gender'] == df['Sub Gender']
        df['Match DOB'] = df['DOB'] == df['Sub DOB']
        df['Match Sum Insured'] = df['Sum Insured'] == df['Sub Sum Insured']
        df['Match UHID'] = df['UHID'] == df['Sub UHID']
        df['Match Sub Active'] = df['Active'] == df['Sub Active']
        df['Match Sub Phone'] = df['Phone'] == df['Sub Phone']
        df['Match Sub Email'] = df['Email'] == df['Sub Email']
        

    
      
 #-------------------------------------------End of Code for Unmatched columns-----------------------------------------------
  
    # Save the data to Excel
    save_to_excel(selected_main_data, Unmatched_main_data, selected_sub_data, unmatch_sub_data, output_path)

    


Processing single sheet 'Sheet1' from file 'MEMBER DETAILS--8_18_2024, 12_40_55 PM.xlsx'
All required columns are present in file 'MEMBER DETAILS--8_18_2024, 12_40_55 PM.xlsx'.
Processing single sheet 'ABHIPROD' from file 'MD_RN_UPGRADE_ABHIPROD_2-81-23-0001567-000_1723788251264.xlsx'


,Master Policy Number,Combi Reference Number,Group Policy Number,Group Policy Holder name,Product Code,GPA Rater Plan,FAMILY_POLICY_NUMBER,Insured Member Name,Policy Start Date,Policy End Date,ABHI Client ID,Endorsement Reference Number,Employee Number,Member Entry Date,Member Exit Date,Relationship,Date of Birth,Age,Gender,Annual Premium without ST /GST,GHI /GPA Sum Insured,CGST 9 %,SGST 9 %,IGST 18%,Total Charged Premium including ST /GST,Status (Active /Inactive),Category,Application Number entry 1,Partner Reference Number 2
0,NaN,NaN,2-81-23-0001567-000,IDEA LAKE INFORMATION TECHNOLOGIES PRIVATE LIM...,5211,NaN,GHI-81-23-5928542-000,DEVISHI TOPKHANE,2023-10-12,2024-01-19 23:59:59,PT98606809,2-81-23-0001567-000-DE-010,101479,2023-10-12,2024-01-19 23:59:59,Son,NaN,9.0,M,0.00,0.0,0.00,0.00,0.0,0.00,CANCELLED,CATEGORY 2,101479,NaN
1,NaN,NaN,2-81-23-0001567-000,IDEA LAKE INFORMATION TECHNOLOGIES PRIVATE LIM...,5211,NaN,GHI-81-23-5928542-000,KUHIKA TOPKHANE,2023-10-12,2024-01-19 23:59:59,PT98606798,2-81-23-0001567-000-DE-010,101479,2023-10-12,2024-01-19 23:59:59,Daughter,NaN,7.0,F,0.00,0.0,0.00,0.00,0.0,0.00,CANCELLED,CATEGORY 2,101479,NaN
2,NaN,NaN,2-81-23-0001567-000,IDEA LAKE INFORMATION TECHNOLOGIES PRIVATE LIM...,5211,NaN,GHI-81-23-5928542-000,PRIYA TOPKHANE,2023-10-12,2024-01-19 23:59:59,PT98606803,2-81-23-0001567-000-DE-010,101479,2023-10-12,2024-01-19 23:59:59,Spouse,19/02/1983,40.0,F,0.00,0.0,0.00,0.00,0.0,0.00,CANCELLED,CATEGORY 2,101479,NaN
3,NaN,NaN,2-81-23-0001567-000,IDEA LAKE INFORMATION TECHNOLOGIES PRIVATE LIM...,5211,NaN,GHI-81-23-5928542-000,ROHIT DINKAR TOPKHANE,2023-10-12,2024-01-19 23:59:59,PT98606791,2-81-23-0001567-000-DE-010,101479,2023-10-12,2024-01-19 23:59:59,Self,11/08/1974,49.0,M,6414.54,NaN,577.31,577.31,NaN,7569.15,CANCELLED,CATEGORY 2,101479,NaN
4,NaN,NaN,2-81-23-0001567-000,IDEA LAKE INFORMATION TECHNOLOGIES PRIVATE LIM...,5211,NaN,GHI-81-23-5928554-000,RAMCHANDRA DADA THENGAL,2023-10-12,2024-02-12 23:59:59,PT98606686,2-81-23-0001567-000-DE-013,101389,2023-10-12,2024-02-12 23:59:59,Spouse,03/02/1991,32.0,M,0.00,0.0,0.00,0.00,0.0,0.00,CANCELLED,CATEGORY 2,101389,NaN



Columns missing in 'MD_RN_UPGRADE_ABHIPROD_2-81-23-0001567-000_1723788251264.xlsx': ['Employee ID', 'Name', 'DOB', 'Coverage Start Date', 'Sum Insured', 'UHID', 'Active', 'Phone', 'Email'].

Available columns:

|| Master Policy Number   || Combi Reference Number   || Group Policy Number   || Group Policy Holder name   || Product Code   || GPA Rater Plan   || FAMILY_POLICY_NUMBER   || Insured Member Name   || Policy Start Date   || Policy End Date   || ABHI Client ID   || Endorsement Reference Number   || Employee Number   || Member Entry Date   || Member Exit Date   || Relationship   || Date of Birth   || Age   || Gender   || Annual Premium without ST /GST   || GHI /GPA Sum Insured   || CGST 9 %   || SGST 9 %   || IGST 18%   || Total Charged Premium including ST /GST   || Status  (Active /Inactive)   || Category   || Application Number entry 1   || Partner Reference Number 2   ||


Please input the column name for 'Employee ID': Employee Number
Please input the column name for 'Name':

C:\Users\Loop\AppData\Local\Temp\ipykernel_4364\3653295848.py:195: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DOB'] = pd.to_datetime(df['DOB']).dt.strftime('%d/%b/%Y')


Unmatched_main_data is empty. No operations to perform.
Recon Data saved to C:\Users\Loop\Desktop\Process Tools\Endo Recon Tool\Output\Recon Data.xlsx
